In [ ]:
!pip install ultralytics
!pip install roboflow
!pip install ruamel.yaml

In [2]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ultralytics
from ultralytics import YOLO
from roboflow import Roboflow
from IPython.display import Image 
from kaggle_secrets import UserSecretsClient
import yaml
from pprint import pprint
import cv2
import os 
import warnings 
warnings.filterwarnings("ignore")
ultralytics.checks()

Ultralytics 8.4.7 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 6634.3/8062.4 GB disk)


# Data inspection and validation

In [3]:
image_sample = cv2.imread("/kaggle/input/smoke-fire-detection-yolo/data/train/images/AoF00005.jpg")
x , y , c = image_sample.shape 
print(f"The dimension af image {y} x {x} and {c} channels")

The dimension af image 1280 x 720 and 3 channels


In [8]:
BASE_PATH = "/kaggle/input/smoke-fire-detection-yolo/data"

splits = ["train", "val", "test"]

def check_split(split):
    img_dir = os.path.join(BASE_PATH, split, "images")
    lbl_dir = os.path.join(BASE_PATH, split, "labels")

    images = [f for f in os.listdir(img_dir) if f.lower().endswith((".jpg", ".png", ".jpeg"))]
    labels = [f for f in os.listdir(lbl_dir) if f.endswith(".txt")]

    img_names = set(os.path.splitext(f)[0] for f in images)
    lbl_names = set(os.path.splitext(f)[0] for f in labels)

    missing_labels = img_names - lbl_names
    missing_images = lbl_names - img_names

    print(f"\n {split}")
    print(f"Images : {len(images)}")
    print(f"Labels : {len(labels)}")

    if not missing_labels and not missing_images:
        print("All images and labels are matched")
    else:
        print(f"Missing labels for {len(missing_labels)} images")
        print(f"Missing Images for {len(missing_images)} labels")

for split in splits:
    check_split(split)


 train
Images : 14122
Labels : 14122
All images and labels are matched

 val
Images : 3099
Labels : 3099
All images and labels are matched

 test
Images : 4306
Labels : 4306
All images and labels are matched


# YAML Cofiguration for YOLO
YOLO uses a Yaml file to define dataset paths and class name 

In [9]:
# original YAML file
path_yaml_file = "/kaggle/input/smoke-fire-detection-yolo/data.yaml"

print("Content of Original Yaml file")
with open(path_yaml_file, 'r') as file:
    data = yaml.safe_load(file)
    print(data)

Content of Original Yaml file
{'path': '/kaggle/working/D Fire Dataset', 'train': 'data/train/images', 'val': 'data/val/images', 'test': 'data/test/images', 'names': ['smoke', 'fire'], 'nc': 2, 'train_count': 14122, 'val_count': 3099, 'test_count': 4306}


In [10]:
# valid path
input_file_path = '/kaggle/input/smoke-fire-detection-yolo/data.yaml'
output_file_path = '/kaggle/working/data.yaml'

with open(input_file_path, 'r') as file:
    data = yaml.safe_load(file)

base_path = '/kaggle/input/smoke-fire-detection-yolo/data'
data['path'] = base_path
data['train'] = f"{base_path}/train/image"
data['val'] = f"{base_path}/val/image"
data['test'] = f"{base_path}/test/image"

with open(output_file_path, 'w') as file:
    yaml.safe_dump(data, file, sort_keys=False)

print(f"Updated YAML saved to: {output_file_path}")
pprint(data)

Updated YAML saved to: /kaggle/working/data.yaml
{'names': ['smoke', 'fire'],
 'nc': 2,
 'path': '/kaggle/input/smoke-fire-detection-yolo/data',
 'test': '/kaggle/input/smoke-fire-detection-yolo/data/test/image',
 'test_count': 4306,
 'train': '/kaggle/input/smoke-fire-detection-yolo/data/train/image',
 'train_count': 14122,
 'val': '/kaggle/input/smoke-fire-detection-yolo/data/val/image',
 'val_count': 3099}


# Checking Dataset Balance
To ensure the model learns effectively, we check the distribution of classes across the dataset splits(train, val, test)

In [ ]:
# To check the Balance Data
from collections import defaultdict

def read_yolo(dataset_metadata):
    train_path = dataset_metadata['train'].replace('images', 'labels')
    val_path = dataset_metadata['val'].replace('images', 'labels')
    test_path = dataset_metadata['test'].replace('images', 'labels')

    split_stats = {
        'train': defaultdict(int),
        'val' : defaultdict(int),
        'test' : defaultdict(int)
    }
    def process_labels(label_dir, split_name):
        if not os.path.exists(label_dir):
            raise FileNotFoundError(f"Labels folder not found: {label_dir}")
            